__Inference set up - Basemodel_Hagen__

In [ ]:
from fastai.vision.all import *
import cv2
import os
import pathlib
import fastai; fastai.__version__

In [ ]:
torch.cuda.set_device(0)

In [ ]:
torch.cuda.current_device(), torch.cuda.get_device_name(0)

In [ ]:
device=torch.device("cuda")

In [ ]:
path = pathlib.Path(os.getcwd())

In [ ]:
path_GT = path/'Daten/Hagen_GT'
path_noisy = path/'Daten/Hagen_noisy'

In [ ]:
%run Dataloader_Hagen.ipynb
%run Perception_loss.ipynb

In [ ]:
dls_den, dblock = get_dls(38)  # required to generate learner, sets batchsize for prediction 

In [ ]:
loss_func = FeatureLoss(vgg_m, blocks[2:5],[1], [1,1,1], [1,1,1]) # dummy

In [ ]:
# define learner w/o learning parameters (e.g. metrics, cbs)
def create_gen_learner():
    return unet_learner(dls_den, bbone, loss_func=loss_func, blur=True, norm_type=NormType.Weight, 
                        self_attention=True, y_range=y_range
                        )

In [ ]:
# create learner
learn_den = create_gen_learner()
# load the Basemodel into that leaner and convert to fp16
learn_den.load('Basemodel_Hagen').to_fp16()

__Set up data and perform inference__

In [ ]:
path_test = path/'Daten/Hagen_testmix/noisy_tiles'
path_preds = path/'Daten/Hagen_testmix/Hagen_pred_tiles'
path_preds.mkdir(exist_ok=True)

In [ ]:
# check path
test_files = get_image_files(path_test, recurse=True)
test_files[0], len(test_files)

In [ ]:
def save_preds(dl, learn):
  "Save away predictions"
  names = dl.dataset.items  
    
  inp, preds, _,decoded = learn.get_preds(dl=dl, with_input=True, with_decoded=True)

  for i,pred in enumerate(preds):      
      arr = pred.clamp_(0,1).permute(1,2,0).numpy()
      arr = arr[:,:,0]*(2**16-1)
      cv2.imwrite(str(path_preds/names[i].name),arr.astype(np.uint16))
  return inp, preds, decoded                  # return outputs for analysis

In [ ]:
"""
test_dl=learn_den.dls.test_dl(test_files)
inp, preds, dec = save_preds(test_dl, learn_den)
"""

In [ ]:
# use this loop with a chunksize, if your RAM cant contain all predictions
chunk_size = 5000
for k in range(0,len(test_files),chunk_size):
    chunk = test_files[k:k+chunk_size]
    test_dl=learn_den.dls.test_dl(chunk)
    inp, preds, dec = save_preds(test_dl, learn_den)